We have 6 classes in this classification task:
    1. barren_land (73396)
    2. building (14922)
    3. grassland (50346)
    4. road (8191)
    5. trees (56808)
    6. water (120331)

As indicated above, they are hugely inbalanced

It's highly likely that a CNN with a classifier at the end will over predict water and under predict road
Since we don't know the natural distribution of these images, we will attempt to balance these images out

How:
    1. Augmentation
    2. Subsampling/oversampling
    3. ???
    

May not be the best practice will need to look up subsampling methods

In [1]:
count_by_cat = {'barren_land' :73396, 'building':14922, 'grassland':50346, 'road':8191, 'trees':56808, 'water':120331}

In [2]:
count_by_cat_perc = {k: v/sum(count_by_cat.values())*100 for k,v in count_by_cat.items()}

In [3]:
count_by_cat_perc

{'barren_land': 22.653505929122144,
 'building': 4.605640845200837,
 'grassland': 15.53917665141947,
 'road': 2.5281332370352536,
 'trees': 17.53365803070427,
 'water': 37.139885306518025}

In [4]:
sum(count_by_cat.values())

323994

Now let's try protyping some of this

In [5]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models, transforms, utils
from PIL import Image
from tqdm import tqdm
from torch.optim import Adam
import math
import cv2

In [26]:
class vgg_preloaded(nn.Module):

	def __init__(self, num_class, use_cuda):
		super(vgg_preloaded, self).__init__()
		self.use_cuda = use_cuda
		self.num_class = num_class
		self.dtype = torch.cuda.FloatTensor if self.use_cuda else torch.FloatTensor
		model = models.inception_v3(pretrained=True)
		self.model = model.cuda() if self.use_cuda else model
		for param in self.model.features.parameters():
			param.require_grad = False
		num_features = self.model.classifier[6].in_features
		features = list(self.model.classifier.children())[:-1] # Remove last layer
		features.extend([nn.Linear(num_features, self.num_class)])
		self.model.classifier = nn.Sequential(*features)
		self.model.classifier.require_grad = True

	def forward(self, inp):
		return(self.model(inp))

In [6]:
image_dir = '../../data/images/'
barren_land_dir = image_dir + 'barren_land/'
barren_land = os.listdir(barren_land_dir)

In [7]:
image_3 = cv2.imread(barren_land_dir + barren_land[3])

In [8]:
image_3.shape

(28, 28, 3)

We need to check if everything is in png format

In [10]:
os.path.isdir(os.path.join(image_dir, os.listdir(image_dir)[0]))

False

In [11]:
# creating individual paths
sub_paths = [os.path.join(image_dir, file_dir) for file_dir in os.listdir(image_dir)]
# check if they're folders
sub_paths = [path for path in sub_paths if os.path.isdir(path)]

In [12]:
sub_paths

['../../data/images/water',
 '../../data/images/trees',
 '../../data/images/road',
 '../../data/images/barren_land',
 '../../data/images/building',
 '../../data/images/grassland']

In [13]:
from tqdm import tqdm
res = [0 if file_name[-3:] == 'png' else 1 for file_name in tqdm(os.listdir(sub_paths[0]))]

100%|██████████| 120332/120332 [00:00<00:00, 1423997.87it/s]


In [14]:
res = {}
for path in sub_paths:
    res[path] = sum([0 if file_name[-3:] == 'png' else 1 for file_name in tqdm(os.listdir(path))])

100%|██████████| 50347/50347 [00:00<00:00, 1360276.11it/s]


In [15]:
res

{'../../data/images/barren_land': 0,
 '../../data/images/building': 0,
 '../../data/images/grassland': 0,
 '../../data/images/road': 0,
 '../../data/images/trees': 0,
 '../../data/images/water': 0}

OK! 

so all our images are of '.png' format

In [16]:
sub_paths

['../../data/images/water',
 '../../data/images/trees',
 '../../data/images/road',
 '../../data/images/barren_land',
 '../../data/images/building',
 '../../data/images/grassland']

In [17]:
class_dict = {'water':1, 'trees':2, 'road':3, 'barren_land': 4, 'building': 5, 'grassland':6}

all_files_dict = {}
for path in sub_paths:
    print(os.path.split(path)[1])
    all_files_dict = {**all_files_dict, **{os.path.join(path, file_name):class_dict[os.path.split(path)[1]] for file_name in  os.listdir(path)}}

water
trees
road
barren_land
building
grassland


In [18]:
len(all_files_dict)

324000

Ok officially we have 32.4k images

In [19]:
set(all_files_dict.values())

{1, 2, 3, 4, 5, 6}

In [20]:
import pandas as pd

In [21]:
test_df = pd.DataFrame.from_dict({'col1': ['a','b','c','d'], 'col2':[1,2,3,4]})

In [22]:
test_df.loc[test_df['col1'] == 'a', 'col2']

0    1
Name: col2, dtype: int64

In [23]:
import numpy as np
nparrayres = np.array(test_df.loc[test_df['col1'] == 'a', 'col2'])
nparrayres

array([1])

In [24]:
torch.from_numpy(nparrayres)[0]

tensor(1)

Now we can prototype this on the .py file

In [25]:
list(all_files_dict.keys())[:15]

['../../data/images/water/59510.png',
 '../../data/images/water/27087.png',
 '../../data/images/water/21069.png',
 '../../data/images/water/20806.png',
 '../../data/images/water/65273.png',
 '../../data/images/water/67526.png',
 '../../data/images/water/110447.png',
 '../../data/images/water/118455.png',
 '../../data/images/water/63757.png',
 '../../data/images/water/73946.png',
 '../../data/images/water/111116.png',
 '../../data/images/water/14412.png',
 '../../data/images/water/19821.png',
 '../../data/images/water/24287.png',
 '../../data/images/water/28949.png']

In [26]:
sorted(list(all_files_dict.keys()))[:15]

['../../data/images/barren_land/0.png',
 '../../data/images/barren_land/1.png',
 '../../data/images/barren_land/10.png',
 '../../data/images/barren_land/100.png',
 '../../data/images/barren_land/1000.png',
 '../../data/images/barren_land/10000.png',
 '../../data/images/barren_land/10001.png',
 '../../data/images/barren_land/10002.png',
 '../../data/images/barren_land/10003.png',
 '../../data/images/barren_land/10004.png',
 '../../data/images/barren_land/10005.png',
 '../../data/images/barren_land/10006.png',
 '../../data/images/barren_land/10007.png',
 '../../data/images/barren_land/10008.png',
 '../../data/images/barren_land/10009.png']

In [44]:
torch.Tensor([0])[0]

tensor(0.)

In [46]:
torch.from_numpy(np.array(0))

tensor(0)

In [ ]:
transforms.RandomVerticalFlip()

In [47]:
?transforms.RandomRotation()

In [58]:
vgg16 = models.vgg16(pretrained=True)

In [61]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [67]:
inceptionv3 = models.inception_v3(pretrained=True)

In [73]:
inceptionv3

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [80]:
inceptionv3.fc.in_features

2048

In [87]:
inceptionv3

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [101]:
for param in inceptionv3.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [82]:
inceptionv3.fc = nn.Linear(2048, 6)

In [86]:
for param in inceptionv3.parameters():
    param.requires_grad = False

In [93]:
inceptionv3.fc.requires_grad = True

In [100]:
for param in inceptionv3.fc.parameters():
    print(param)
    param.requires_grad = True
    print(param.requires_grad)

Parameter containing:
tensor([[ 1.7090e-02,  4.2657e-03, -1.9937e-02,  ..., -1.0503e-02,
         -3.3944e-03, -1.2688e-02],
        [ 2.1957e-02,  6.2939e-03, -1.5725e-02,  ...,  1.2078e-02,
          2.0476e-03, -6.2216e-03],
        [ 1.5741e-03,  1.2009e-02,  1.7172e-02,  ..., -3.9985e-03,
          9.9693e-03, -1.1624e-02],
        [-6.8041e-03, -4.1619e-03,  6.6067e-03,  ..., -1.1594e-02,
         -4.2634e-03, -9.2580e-03],
        [-3.6699e-03, -5.1785e-03, -2.1240e-03,  ...,  1.2855e-02,
         -1.1991e-03,  1.3751e-02],
        [-4.6903e-03, -6.1966e-03,  7.8006e-03,  ..., -1.6699e-02,
         -1.2975e-02,  9.3672e-03]])
True
Parameter containing:
tensor([ 0.0094,  0.0027, -0.0163, -0.0023, -0.0134, -0.0141])
True


In [102]:
inceptionv3.train()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [104]:
from torch.optim import SGD